In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
import sys
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from skimage.color import rgb2lab
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC



OUTPUT_TEMPLATE = (
    'SVM classifier:      {svm_rgb:.3g}\n'
    'kNN classifier:      {knn_rgb:.3g} {knn_lab:.3g}\n'
)


# representative RGB colours for each label, for nice display
COLOUR_RGB = {
    'red': (255, 0, 0),
    'orange': (255, 114, 0),
    'yellow': (255, 255, 0),
    'green': (0, 230, 0),
    'blue': (0, 0, 255),
    'purple': (187, 0, 187),
    'brown': (117, 60, 0),
    'black': (0, 0, 0),
    'grey': (150, 150, 150),
    'white': (255, 255, 255),
}
name_to_rgb = np.vectorize(COLOUR_RGB.get, otypes=[np.uint8, np.uint8, np.uint8])


In [56]:
def plot_predictions(model, lum=71, resolution=256):
    """
    Create a slice of LAB colour space with given luminance; predict with the model; plot the results.
    """
    wid = resolution
    hei = resolution
    n_ticks = 5

    # create a hei*wid grid of LAB colour values, with L=lum
    ag = np.linspace(-100, 100, wid)
    bg = np.linspace(-100, 100, hei)
    aa, bb = np.meshgrid(ag, bg)
    ll = lum * np.ones((hei, wid))
    lab_grid = np.stack([ll, aa, bb], axis=2)

    # convert to RGB for consistency with original input
    X_grid = lab2rgb(lab_grid)

    # predict and convert predictions to colours so we can see what's happening
    y_grid = model.predict(X_grid.reshape((wid*hei, 3)))
    pixels = np.stack(name_to_rgb(y_grid), axis=1) / 255
    pixels = pixels.reshape((hei, wid, 3))

    # plot input and predictions
    plt.figure(figsize=(10, 5))
    plt.suptitle('Predictions at L=%g' % (lum,))
    plt.subplot(1, 2, 1)
    plt.title('Inputs')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.ylabel('B')
    plt.imshow(X_grid.reshape((hei, wid, 3)))

    plt.subplot(1, 2, 2)
    plt.title('Predicted Labels')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.imshow(pixels)


In [57]:
def getLabColor(X):
    X2d = X[:, np.newaxis]
    X3d = X2d[:, np.newaxis]
    XLab3d = rgb2lab(X3d)
    return np.reshape(XLab3d, (len(X), 3))

In [58]:
# data = pd.read_csv(sys.argv[1])
data = pd.read_csv("colour-data.csv")
X = data[['R', 'G', 'B']].values / 255
y = data['Label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [59]:
svc_rgb_model = SVC(kernel='linear', C=1000)


In [60]:
svc_rgb_model.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [61]:
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma' : [1e-3, 1e-4],
                     'C'     : [1, 10, 100, 1000]
                    },
                    {'kernel': ['linear'], 
                     'C'     : [1, 10, 100, 1000]
                    }
                   ]

In [62]:
clf = GridSearchCV(svc_rgb_model, tuned_parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)

{'C': 1000, 'kernel': 'linear'}


In [63]:
knn_lab_model = make_pipeline(
    FunctionTransformer(getLabColor),
    KNeighborsClassifier(n_neighbors=7)
)
knn_rgb_model = KNeighborsClassifier(n_neighbors=7)

In [64]:
knnparams = [{'n_neighbors': list(range(1, 50))}]

In [65]:
clf1 = GridSearchCV(knn_rgb_model, knnparams)
clf1.fit(X_train, y_train)
print(clf1.best_params_)

{'n_neighbors': 10}


In [69]:
knn_rgb_model = KNeighborsClassifier(n_neighbors=10)
knn_rgb_model.fit(X,y)
knn_rgb_model.score(X_test, y_test)

0.75493421052631582